In [ ]:
%pip install joblib


In [ ]:
import pandas as pd
import os
import re
import joblib
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
import numpy as np
from sklearn.feature_selection import RFECV
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to the folder containing the CSV files
folder_path = '/content/drive/Shareddrives/Bierdify/dtl-csv-raw'

In [ ]:

# Read the formatted labels CSV file
labels_csv_path = '/content/drive/Shareddrives/Bierdify/dtl_csv.csv'
labels_df = pd.read_csv(labels_csv_path)

# Iterate through each CSV file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('_dtl.csv'):  # Adjust this pattern to match your file naming convention
        video_number = int(filename.split('_')[0])  # Extracting the video number from the filename
        csv_file_path = os.path.join(folder_path, filename)
        video_df = pd.read_csv(csv_file_path)

        # Add the 'event' column with a default value of 4
        video_df['event'] = 4

        # Filter the labels for the current video
        filtered_labels_df = labels_df[labels_df['video_number'] == video_number]

        # Merge and update the 'event' column
        merged_df = video_df.merge(filtered_labels_df, left_on='frame_index', right_on='frame', how='left')
        merged_df['event'] = merged_df['label'].fillna(merged_df['event']).astype(int)
        merged_df.drop(columns=['video_number', 'frame', 'label'], inplace=True)

        # Save the modified DataFrame
        modified_file_path = os.path.join(folder_path, 'modified_' + filename)
        merged_df.to_csv(modified_file_path, index=False)
        print(f"Processed and saved modified CSV: {modified_file_path}")





Processed and saved modified CSV: /content/drive/Shareddrives/Bierdify/dtl-csv-raw/modified_47_dtl.csv
Processed and saved modified CSV: /content/drive/Shareddrives/Bierdify/dtl-csv-raw/modified_46_dtl.csv
Processed and saved modified CSV: /content/drive/Shareddrives/Bierdify/dtl-csv-raw/modified_36_dtl.csv
Processed and saved modified CSV: /content/drive/Shareddrives/Bierdify/dtl-csv-raw/modified_54_dtl.csv


ParserError: ignored

#ModelXGBOOST


In [ ]:
# Path to the folder containing your CSV files
folder_path = '/content/drive/Shareddrives/Bierdify/just-for-testing-csv'

In [ ]:


# Initialize an empty DataFrame to hold all the data
combined_df = pd.DataFrame()

# Loop through each file in the folder and append it to the combined DataFrame
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # Check if the file is a CSV
        file_path = os.path.join(folder_path, filename)
        temp_df = pd.read_csv(file_path)

        # Extract video ID from the filename (assuming filename without extension is the video ID)
        video_id = filename.split('.')[0]  # This splits the filename at the dot and takes the first part
        temp_df['video ID'] = video_id  # Add a new column with the video ID

        combined_df = pd.concat([combined_df, temp_df])








In [ ]:
# Ensure no index duplication
combined_df.reset_index(drop=True, inplace=True)

# Data preprocessing steps (if needed)
# ...





In [ ]:
# Remapping the 'event' label
label_mapping = {0: 0, 1: 1, 2: 2, 4: 3}
combined_df['event'] = combined_df['event'].map(label_mapping)

# Then proceed with your model training as before


In [ ]:
# Function to extract coordinates
def extract_coordinates(coord_string):
    coord_string = str(coord_string)
    numbers = re.findall(r'[-+]?[0-9]*\.?[0-9]+', coord_string)
    # Ensure that three elements are always returned
    return [float(numbers[i]) if i < len(numbers) else None for i in range(3)]

# Process each coordinate column
for i in range(17):  # Adjust this range based on the number of bbox_0_coord columns
    col_name = f'bbox_0_coord_{i}'
    combined_df[[f'{col_name}_x', f'{col_name}_y', f'{col_name}_z']] = combined_df[col_name].apply(lambda x: pd.Series(extract_coordinates(x)))


In [ ]:
combined_df

,frame_index,bbox_0_coord_0,bbox_0_coord_1,bbox_0_coord_2,bbox_0_coord_3,bbox_0_coord_4,bbox_0_coord_5,bbox_0_coord_6,bbox_0_coord_7,bbox_0_coord_8,...,bbox_0_coord_13_z,bbox_0_coord_14_x,bbox_0_coord_14_y,bbox_0_coord_14_z,bbox_0_coord_15_x,bbox_0_coord_15_y,bbox_0_coord_15_z,bbox_0_coord_16_x,bbox_0_coord_16_y,bbox_0_coord_16_z
0,0,[94.75 68.1875 0.29052734],[9.487500e+01 6.618750e+01 6.756592e-02],[94.875 66.6875 0.32128906],[86.5625 63.8125 0.14782715],[92.25 66.6875 0.5756836],[81.25 69.625 0.6582031],[85.5 74.25 0.90478516],[85.875 77.875 0.25268555],[84.6875 90.6875 0.8989258],...,0.660156,77.8125,117.6875,0.812988,79.8125,133.0000,0.657715,76.1250,138.125,0.671387
1,1,[95.75 67.625 0.29736328],[9.56875e+01 6.55000e+01 5.92041e-02],[95.75 66.125 0.32641602],[86.5625 63.34375 0.13586426],[92.75 66.5 0.5888672],[81.1875 69.625 0.6533203],[85.625 74.3125 0.90722656],[84. 79.4375 0.23547363],[84.5625 90.6875 0.8989258],...,0.662109,77.8125,117.8125,0.816895,79.8125,133.1250,0.650391,76.1875,138.125,0.663574
2,2,[94.4375 68.625 0.30126953],[9.468750e+01 6.675000e+01 8.312988e-02],[94.625 66.9375 0.32373047],[86.6875 64.375 0.19616699],[91.875 66.5 0.53027344],[80.8125 70.6875 0.6479492],[85.125 73.875 0.8901367],[79.6875 86.0625 0.19433594],[84.75 90.6875 0.8383789],...,0.652344,77.4375,118.1250,0.810059,79.6250,133.2500,0.619629,76.6250,137.750,0.652344
3,3,[94.5 67.875 0.2758789],[9.487500e+01 6.581250e+01 7.104492e-02],[94.6875 66.125 0.30273438],[86.875 63.5 0.1427002],[92.0625 65.9375 0.5527344],[81.4375 69.75 0.6308594],[84.6875 73.9375 0.88183594],[84.9375 80.1875 0.2368164],[84.1875 90.75 0.8642578],...,0.625000,77.8125,117.9375,0.808105,79.8125,133.2500,0.642578,76.2500,138.125,0.689453
4,4,[94.875 64. 0.3269043],[95. 62.3125 0.14013672],[94.5 62.3125 0.31396484],[88.4375 62.15625 0.16772461],[91.75 63.3125 0.4465332],[83.5 70.0625 0.6020508],[84.8125 73.5 0.8676758],[91.5625 72.875 0.2890625],[84. 90.875 0.8432617],...,0.640625,78.0625,118.4375,0.794922,79.0000,134.0000,0.659180,75.6875,137.750,0.668945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111637,344,[80.4375 46.625 0.24865723],[81.125 44.78125 0.2697754],[82.125 45. 0.11199951],[79.375 44.8125 0.6123047],[87.875 45.875 0.41577148],[71.625 54.125 0.9272461],[88.3125 54.875 0.87353516],[63.375 66.125 0.68603516],[86.5 58.4375 0.42919922],...,0.919922,84.0625,101.8750,0.918945,76.5000,125.7500,0.829102,90.7500,123.250,0.840820
111638,345,[79.4375 46.46875 0.17272949],[80.25 44.5 0.1821289],[8.118750e+01 4.478125e+01 6.805420e-02],[79.375 44.75 0.5859375],[88. 46.0625 0.4416504],[71.5625 54.3125 0.9458008],[88.375 55.0625 0.87402344],[64.0625 67.5 0.7993164],[87.0625 57.21875 0.3840332],...,0.928711,85.0000,102.2500,0.926758,76.2500,126.5625,0.852051,91.8750,123.750,0.868164
111639,346,[78.8125 46.0625 0.16259766],[79.6875 44.28125 0.17077637],[80.5 44.5625 0.08239746],[79.4375 44.3125 0.640625],[87.75 45.6875 0.4638672],[71.375 54.3125 0.94921875],[88.6875 54.8125 0.87939453],[63.65625 67.125 0.7944336],[88. 59.40625 0.4272461],...,0.903809,84.3750,102.0000,0.870605,76.5000,126.5000,0.804199,91.1250,123.875,0.796387
111640,347,[79. 46.15625 0.15710449],[79.875 44.1875 0.1652832],[8.081250e+01 4.450000e+01 6.097412e-02],[79.5 44.59375 0.5678711],[88.1875 45.8125 0.4416504],[72.0625 54.3125 0.9506836],[88.875 55.125 0.8613281],[63.9375 67.75 0.8457031],[87.5 57.875 0.35009766],...,0.927246,84.8750,102.2500,0.911133,76.3125,126.7500,0.844727,91.8125,123.875,0.847656


In [ ]:
# Optionally, drop the original coordinate columns
combined_df.drop([f'bbox_0_coord_{i}' for i in range(17)], axis=1, inplace=True)


In [ ]:
# Dropping the z coordinate columns
for i in range(17):  # Adjust this range based on the number of bbox_0_coord columns
    z_col_name = f'bbox_0_coord_{i}_z'
    if z_col_name in combined_df.columns:
        combined_df.drop(z_col_name, axis=1, inplace=True)

In [ ]:
# Calculating relative positions
combined_df['hand_to_head_x'] = combined_df['bbox_0_coord_4_x'] - combined_df['bbox_0_coord_0_x']
combined_df['hand_to_head_y'] = combined_df['bbox_0_coord_4_y'] - combined_df['bbox_0_coord_0_y']

# Calculating Euclidean distance (optional)
combined_df['hand_head_distance'] = ((combined_df['hand_to_head_x']**2 + combined_df['hand_to_head_y']**2)**0.5)

In [ ]:
combined_df

,frame_index,event,video ID,bbox_0_coord_0_x,bbox_0_coord_0_y,bbox_0_coord_1_x,bbox_0_coord_1_y,bbox_0_coord_2_x,bbox_0_coord_2_y,bbox_0_coord_3_x,...,bbox_0_coord_13_y,bbox_0_coord_14_x,bbox_0_coord_14_y,bbox_0_coord_15_x,bbox_0_coord_15_y,bbox_0_coord_16_x,bbox_0_coord_16_y,hand_to_head_x,hand_to_head_y,hand_head_distance
0,0,3,modified_47_dtl,94.7500,68.18750,9.48750,1.00000,94.87500,66.68750,86.5625,...,116.0625,77.8125,117.6875,79.8125,133.0000,76.1250,138.125,-2.5000,-1.50000,2.915476
1,1,3,modified_47_dtl,95.7500,67.62500,9.56875,1.00000,95.75000,66.12500,86.5625,...,116.1875,77.8125,117.8125,79.8125,133.1250,76.1875,138.125,-3.0000,-1.12500,3.204001
2,2,3,modified_47_dtl,94.4375,68.62500,9.46875,1.00000,94.62500,66.93750,86.6875,...,116.6250,77.4375,118.1250,79.6250,133.2500,76.6250,137.750,-2.5625,-2.12500,3.328968
3,3,3,modified_47_dtl,94.5000,67.87500,9.48750,1.00000,94.68750,66.12500,86.8750,...,116.3750,77.8125,117.9375,79.8125,133.2500,76.2500,138.125,-2.4375,-1.93750,3.113730
4,4,3,modified_47_dtl,94.8750,64.00000,95.00000,62.31250,94.50000,62.31250,88.4375,...,117.0625,78.0625,118.4375,79.0000,134.0000,75.6875,137.750,-3.1250,-0.68750,3.199731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111637,344,3,modified_255_dtl,80.4375,46.62500,81.12500,44.78125,82.12500,45.00000,79.3750,...,103.5000,84.0625,101.8750,76.5000,125.7500,90.7500,123.250,7.4375,-0.75000,7.475219
111638,345,3,modified_255_dtl,79.4375,46.46875,80.25000,44.50000,8.11875,1.00000,79.3750,...,104.0000,85.0000,102.2500,76.2500,126.5625,91.8750,123.750,8.5625,-0.40625,8.572132
111639,346,3,modified_255_dtl,78.8125,46.06250,79.68750,44.28125,80.50000,44.56250,79.4375,...,103.6875,84.3750,102.0000,76.5000,126.5000,91.1250,123.875,8.9375,-0.37500,8.945364
111640,347,3,modified_255_dtl,79.0000,46.15625,79.87500,44.18750,8.08125,1.00000,79.5000,...,104.0000,84.8750,102.2500,76.3125,126.7500,91.8125,123.875,9.1875,-0.34375,9.193928


In [ ]:

# Calculate the relative distance between right hand and right ankle
combined_df['relative_distance'] = np.sqrt((combined_df['bbox_0_coord_4_x'] - combined_df['bbox_0_coord_10_x']) ** 2 + (combined_df['bbox_0_coord_4_y'] - combined_df['bbox_0_coord_10_y']) ** 2)


In [ ]:

# Calculate displacement for right and left hands in each frame
combined_df['right_hand_movement'] = np.sqrt(combined_df['bbox_0_coord_4_x'].diff() ** 2 + combined_df['bbox_0_coord_4_y'].diff() ** 2)
combined_df['left_hand_movement'] = np.sqrt(combined_df['bbox_0_coord_7_x'].diff() ** 2 + combined_df['bbox_0_coord_7_y'].diff() ** 2)

# Replace NaN values (first row) with 0
combined_df['right_hand_movement'].fillna(0, inplace=True)
combined_df['left_hand_movement'].fillna(0, inplace=True)

# Apply a rolling window of 100 frames and find the max movement
combined_df['max_right_hand_movement'] = combined_df['right_hand_movement'].rolling(window=100, min_periods=1).max()
combined_df['max_left_hand_movement'] = combined_df['left_hand_movement'].rolling(window=100, min_periods=1).max()

# Mark the frames where the hand movement is the maximum within the 100 frame window
combined_df['right_hand_max_movement_frame'] = combined_df['right_hand_movement'] == combined_df['max_right_hand_movement']
combined_df['left_hand_max_movement_frame'] = combined_df['left_hand_movement'] == combined_df['max_left_hand_movement']


In [ ]:
# Calculate the displacement for each hand
combined_df['right_hand_displacement'] = np.sqrt((combined_df['bbox_0_coord_4_x'].diff() ** 2) + (combined_df['bbox_0_coord_4_y'].diff() ** 2))
combined_df['left_hand_displacement'] = np.sqrt((combined_df['bbox_0_coord_7_x'].diff() ** 2) + (combined_df['bbox_0_coord_7_y'].diff() ** 2))

# Calculate the sum of displacements over 20 frames
combined_df['right_hand_movement_20_frames'] = combined_df['right_hand_displacement'].rolling(window=20, min_periods=1).sum()
combined_df['left_hand_movement_20_frames'] = combined_df['left_hand_displacement'].rolling(window=20, min_periods=1).sum()


In [ ]:
# Assume constant time interval between frames (e.g., frame_rate = 1/frame_rate seconds per frame)
frame_rate = 30  # Adjust this based on your actual frame rate
time_interval = 1 / frame_rate

# Calculate the sum of displacements over 10 frames for each hand
combined_df['right_hand_movement_10_frames'] = combined_df['right_hand_displacement'].rolling(window=10, min_periods=1).sum()
combined_df['left_hand_movement_10_frames'] = combined_df['left_hand_displacement'].rolling(window=10, min_periods=1).sum()

# Calculate speed (distance/time) over a 10-frame window for each hand
combined_df['right_hand_speed_10_frames'] = combined_df['right_hand_movement_10_frames'] / (10 * time_interval)
combined_df['left_hand_speed_10_frames'] = combined_df['left_hand_movement_10_frames'] / (10 * time_interval)

In [ ]:
combined_df

,frame_index,event,video ID,bbox_0_coord_0_x,bbox_0_coord_0_y,bbox_0_coord_1_x,bbox_0_coord_1_y,bbox_0_coord_2_x,bbox_0_coord_2_y,bbox_0_coord_3_x,...,right_hand_max_movement_frame,left_hand_max_movement_frame,right_hand_displacement,left_hand_displacement,right_hand_movement_20_frames,left_hand_movement_20_frames,right_hand_movement_10_frames,left_hand_movement_10_frames,right_hand_speed_10_frames,left_hand_speed_10_frames
0,0,3,modified_47_dtl,94.7500,68.18750,9.48750,1.00000,94.87500,66.68750,86.5625,...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3,modified_47_dtl,95.7500,67.62500,9.56875,1.00000,95.75000,66.12500,86.5625,...,True,True,0.534000,2.440703,0.534000,2.440703,0.534000,2.440703,1.602001,7.322109
2,2,3,modified_47_dtl,94.4375,68.62500,9.46875,1.00000,94.62500,66.93750,86.6875,...,True,True,0.875000,7.904953,1.409000,10.345656,1.409000,10.345656,4.227001,31.036968
3,3,3,modified_47_dtl,94.5000,67.87500,9.48750,1.00000,94.68750,66.12500,86.8750,...,False,False,0.592927,7.878967,2.001927,18.224623,2.001927,18.224623,6.005782,54.673870
4,4,3,modified_47_dtl,94.8750,64.00000,95.00000,62.31250,94.50000,62.31250,88.4375,...,True,True,2.643536,9.867283,4.645463,28.091907,4.645463,28.091907,13.936389,84.275720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111637,344,3,modified_255_dtl,80.4375,46.62500,81.12500,44.78125,82.12500,45.00000,79.3750,...,False,False,0.475986,0.562500,13.894323,19.525230,2.260750,6.461747,6.782250,19.385242
111638,345,3,modified_255_dtl,79.4375,46.46875,80.25000,44.50000,8.11875,1.00000,79.3750,...,False,False,0.225347,1.537297,5.936942,16.679276,2.346343,7.773697,7.039029,23.321091
111639,346,3,modified_255_dtl,78.8125,46.06250,79.68750,44.28125,80.50000,44.56250,79.4375,...,False,False,0.450694,0.552869,5.377918,16.281713,2.708648,7.613956,8.125945,22.841869
111640,347,3,modified_255_dtl,79.0000,46.15625,79.87500,44.18750,8.08125,1.00000,79.5000,...,False,False,0.455007,0.685366,5.607577,15.563610,3.023901,7.934888,9.071703,23.804663


In [ ]:
# Assuming combined_df is your loaded DataFrame

# Group by video ID and get unique IDs
video_ids = combined_df['video ID'].unique()

# Split video IDs into train and test sets (80/20 split)
train_ids, test_ids = train_test_split(video_ids, test_size=0.20, random_state=42)

# Create training and test datasets
train_df = combined_df[combined_df['video ID'].isin(train_ids)]
test_df = combined_df[combined_df['video ID'].isin(test_ids)]

# Now you have train_df and test_df as your training and testing sets

In [ ]:
train_df

,frame_index,event,video ID,bbox_0_coord_0_x,bbox_0_coord_0_y,bbox_0_coord_1_x,bbox_0_coord_1_y,bbox_0_coord_2_x,bbox_0_coord_2_y,bbox_0_coord_3_x,...,right_hand_displacement,left_hand_displacement,right_hand_movement_20_frames,left_hand_movement_20_frames,right_hand_movement,left_hand_movement,max_right_hand_movement,max_left_hand_movement,right_hand_max_movement_frame,left_hand_max_movement_frame
403,0,3,modified_46_dtl,108.3750,61.00000,108.7500,58.28125,108.68750,59.00000,100.062500,...,12.084501,17.549595,42.028124,74.508452,12.084501,17.549595,23.241010,33.342037,False,False
404,1,3,modified_46_dtl,108.9375,60.93750,109.3750,58.31250,109.12500,58.75000,101.375000,...,0.406250,0.576222,40.176575,74.897174,0.406250,0.576222,21.012738,33.342037,False,False
405,2,3,modified_46_dtl,108.6250,61.18750,109.3750,58.50000,108.62500,58.81250,1.018750,...,0.950432,1.790819,40.762573,76.098369,0.950432,1.790819,21.012738,33.342037,False,False
406,3,3,modified_46_dtl,108.3750,61.21875,109.0000,58.53125,108.43750,58.81250,101.687500,...,0.209631,3.447146,40.240660,79.233014,0.209631,3.447146,21.012738,33.342037,False,False
407,4,3,modified_46_dtl,108.3750,61.00000,109.0000,58.37500,108.31250,58.68750,1.016875,...,0.244070,2.601082,39.437040,81.636453,0.244070,2.601082,21.012738,33.342037,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111637,344,3,modified_255_dtl,80.4375,46.62500,81.1250,44.78125,82.12500,45.00000,79.375000,...,0.475986,0.562500,13.894323,19.525230,0.475986,0.562500,8.182728,10.568416,False,False
111638,345,3,modified_255_dtl,79.4375,46.46875,80.2500,44.50000,8.11875,1.00000,79.375000,...,0.225347,1.537297,5.936942,16.679276,0.225347,1.537297,8.182728,10.568416,False,False
111639,346,3,modified_255_dtl,78.8125,46.06250,79.6875,44.28125,80.50000,44.56250,79.437500,...,0.450694,0.552869,5.377918,16.281713,0.450694,0.552869,8.182728,10.568416,False,False
111640,347,3,modified_255_dtl,79.0000,46.15625,79.8750,44.18750,8.08125,1.00000,79.500000,...,0.455007,0.685366,5.607577,15.563610,0.455007,0.685366,8.182728,10.568416,False,False


In [ ]:
test_df

,frame_index,event,video ID,bbox_0_coord_0_x,bbox_0_coord_0_y,bbox_0_coord_1_x,bbox_0_coord_1_y,bbox_0_coord_2_x,bbox_0_coord_2_y,bbox_0_coord_3_x,...,bbox_0_coord_16_x,bbox_0_coord_16_y,hand_to_head_x,hand_to_head_y,hand_head_distance,relative_distance,right_hand_displacement,left_hand_displacement,right_hand_movement_20_frames,left_hand_movement_20_frames
0,0,3,modified_47_dtl,94.7500,68.187500,9.48750,1.000000,94.8750,66.687500,86.5625,...,76.1250,138.125,-2.5000,-1.500000,2.915476,31.819928,NaN,NaN,NaN,NaN
1,1,3,modified_47_dtl,95.7500,67.625000,9.56875,1.000000,95.7500,66.125000,86.5625,...,76.1875,138.125,-3.0000,-1.125000,3.204001,32.063475,0.534000,2.440703,0.534000,2.440703
2,2,3,modified_47_dtl,94.4375,68.625000,9.46875,1.000000,94.6250,66.937500,86.6875,...,76.6250,137.750,-2.5625,-2.125000,3.328968,32.019769,0.875000,7.904953,1.409000,10.345656
3,3,3,modified_47_dtl,94.5000,67.875000,9.48750,1.000000,94.6875,66.125000,86.8750,...,76.2500,138.125,-2.4375,-1.937500,3.113730,32.577851,0.592927,7.878967,2.001927,18.224623
4,4,3,modified_47_dtl,94.8750,64.000000,95.00000,62.312500,94.5000,62.312500,88.4375,...,75.6875,137.750,-3.1250,-0.687500,3.199731,35.225002,2.643536,9.867283,4.645463,28.091907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110449,529,3,modified_45_dtl,80.3125,28.906250,78.06250,27.218750,82.0625,27.328125,70.8125,...,74.3125,139.375,4.7500,-1.031250,4.860656,14.087339,8.910703,3.459165,82.420381,128.817475
110450,530,3,modified_45_dtl,84.5000,30.437500,82.31250,28.500000,85.3125,29.109375,72.9375,...,74.8125,139.500,0.6250,-0.625000,0.883883,16.153197,1.938508,0.742804,83.249183,126.611332
110451,531,3,modified_45_dtl,84.6250,28.984375,82.37500,27.078125,85.3750,27.453125,72.5625,...,74.8750,140.250,0.0000,-1.375000,1.375000,24.172562,2.259150,1.613239,84.473774,126.778604
110452,532,3,modified_45_dtl,82.6875,26.390625,80.87500,24.531250,83.8125,24.578125,71.9375,...,74.9375,139.125,0.7500,-0.828125,1.117269,22.361117,2.366401,0.731544,86.340175,127.300516


In [ ]:
weights = {
    'hand_to_head_x': 1.0,
    'hand_to_head_y': 1.0,
    'hand_head_distance': 1.0,
    'relative_distance': 2.0,
    'right_hand_displacement': 3.0,
    'left_hand_displacement': 3.0,
    'right_hand_movement_20_frames': 6.0,
    'left_hand_movement_20_frames': 6.0,
    'max_right_hand_movement':3.0,
    'max_left_hand_movement':	3.0,
    'right_hand_max_movement_frame': 5.0,
    'left_hand_max_movement_frame': 5.0,
    'right_hand_movement_10_frames': 3.0,
    'left_hand_movement_10_frames':	3.0,
    'right_hand_speed_10_frames': 7.0,
    'left_hand_speed_10_frames': 7.0,
}


In [ ]:
X_train = train_df.drop(['event', 'video ID'], axis=1)  # Drop 'event' and 'video ID' for training data
X_test = test_df.drop(['event', 'video ID'], axis=1)    # Drop 'event' and 'video ID' for testing data
y_train = train_df['event']
y_test = test_df['event']






In [ ]:
# Calculate the weighted sum for training data
X_train['weighted_sum'] = np.sum([X_train[col] * weights.get(col, 1.0) for col in X_train.columns], axis=0)

# Calculate the weighted sum for testing data
X_test['weighted_sum'] = np.sum([X_test[col] * weights.get(col, 1.0) for col in X_test.columns], axis=0)

In [ ]:
X_train = X_train.select_dtypes(include=['int', 'float', 'bool'])
X_test = X_test.select_dtypes(include=['int', 'float', 'bool'])

In [ ]:
train_df

,frame_index,event,video ID,bbox_0_coord_0_x,bbox_0_coord_0_y,bbox_0_coord_1_x,bbox_0_coord_1_y,bbox_0_coord_2_x,bbox_0_coord_2_y,bbox_0_coord_3_x,...,right_hand_max_movement_frame,left_hand_max_movement_frame,right_hand_displacement,left_hand_displacement,right_hand_movement_20_frames,left_hand_movement_20_frames,right_hand_movement_10_frames,left_hand_movement_10_frames,right_hand_speed_10_frames,left_hand_speed_10_frames
403,0,3,modified_46_dtl,108.3750,61.00000,108.7500,58.28125,108.68750,59.00000,100.062500,...,False,False,12.084501,17.549595,42.028124,74.508452,31.949047,70.267862,95.847140,210.803585
404,1,3,modified_46_dtl,108.9375,60.93750,109.3750,58.31250,109.12500,58.75000,101.375000,...,False,False,0.406250,0.576222,40.176575,74.897174,31.290960,69.691640,93.872880,209.074920
405,2,3,modified_46_dtl,108.6250,61.18750,109.3750,58.50000,108.62500,58.81250,1.018750,...,False,False,0.950432,1.790819,40.762573,76.098369,31.067123,70.355724,93.201370,211.067172
406,3,3,modified_46_dtl,108.3750,61.21875,109.0000,58.53125,108.43750,58.81250,101.687500,...,False,False,0.209631,3.447146,40.240660,79.233014,26.251040,54.200574,78.753119,162.601721
407,4,3,modified_46_dtl,108.3750,61.00000,109.0000,58.37500,108.31250,58.68750,1.016875,...,False,False,0.244070,2.601082,39.437040,81.636453,24.332529,50.915695,72.997586,152.747084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111637,344,3,modified_255_dtl,80.4375,46.62500,81.1250,44.78125,82.12500,45.00000,79.375000,...,False,False,0.475986,0.562500,13.894323,19.525230,2.260750,6.461747,6.782250,19.385242
111638,345,3,modified_255_dtl,79.4375,46.46875,80.2500,44.50000,8.11875,1.00000,79.375000,...,False,False,0.225347,1.537297,5.936942,16.679276,2.346343,7.773697,7.039029,23.321091
111639,346,3,modified_255_dtl,78.8125,46.06250,79.6875,44.28125,80.50000,44.56250,79.437500,...,False,False,0.450694,0.552869,5.377918,16.281713,2.708648,7.613956,8.125945,22.841869
111640,347,3,modified_255_dtl,79.0000,46.15625,79.8750,44.18750,8.08125,1.00000,79.500000,...,False,False,0.455007,0.685366,5.607577,15.563610,3.023901,7.934888,9.071703,23.804663


In [ ]:
class_weights = y_train.value_counts().max() / y_train.value_counts()
weights = y_train.map(class_weights).values

In [ ]:

# Train the model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train, sample_weight=weights)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

TypeError: ignored

In [ ]:
# Train the model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8958612716763006
              precision    recall  f1-score   support

           0       0.80      0.73      0.76      2539
           1       0.80      0.67      0.73       485
           2       0.79      0.76      0.77      2437
           3       0.93      0.95      0.94     16164

    accuracy                           0.90     21625
   macro avg       0.83      0.78      0.80     21625
weighted avg       0.89      0.90      0.89     21625



In [ ]:
def calculate_average_frame_counts(combined_df):
    # Group by video ID and event, then count frames
    frame_counts = combined_df.groupby(['video ID', 'event']).size().reset_index(name='counts')

    # Calculate average frame count for each event across all videos
    average_frame_counts = frame_counts.groupby('event')['counts'].mean().to_dict()

    return average_frame_counts


In [ ]:
averages = calculate_average_frame_counts(combined_df)

In [ ]:
def refine_predictions(predictions, averages):
    refined = predictions.copy()
    avg_0 = int(averages.get(0, 20))  # Default to 20 if average not found
    avg_2 = int(averages.get(2, 30))  # Default to 30 if average not found

    for i in range(1, len(predictions) - 1):
        if predictions[i] == 1 and predictions[i + 1] == 1:
            # Convert the frame before to 1
            refined[i - 1] = 1

            # Convert the previous frames to 0
            start_index = max(i - 1 - avg_0, 0)
            refined[start_index:i - 1] = [0] * (i - 1 - start_index)

            # Convert the next frame to 1 and the following to 2
            end_index = min(i + 2 + avg_2, len(predictions))
            refined[i + 2:end_index] = [2] * (end_index - i - 2)

    return refined


In [ ]:
def refine_predictions_with_label_1_rule(predictions):
    refined = predictions.copy()

    # Iterate over the predictions
    i = 0
    while i < len(refined) - 4:
        if refined[i] == 0:
            # Convert the relevant part of the array to a list to use the index method
            refined_list = list(refined[i:])

            try:
                # Find the index of the next 2 after the last 0
                next_2_index = refined_list.index(2)
            except ValueError:
                break  # No more 2's found, exit loop

            last_0_index = i + refined_list[:next_2_index].count(0) - 1
            first_2_index = last_0_index + 1 + refined_list[last_0_index + 1:next_2_index].count(1)

            # Ensure there are at least four 1's between the last 0 and the first 2
            num_ones_needed = 4 - (first_2_index - last_0_index - 1)
            if num_ones_needed > 0:
                refined[last_0_index + 1:last_0_index + 1 + num_ones_needed] = [1] * num_ones_needed

            i = first_2_index + i  # Continue from the first 2 index
        else:
            i += 1

    return refined



In [ ]:
def combined_refinement(predictions, averages):
    # First, apply the label 1 specific rule
    refined = refine_predictions_with_label_1_rule(predictions)

    # Then, apply the refinement based on averages
    refined = refine_predictions(refined, averages)

    return refined

In [ ]:
y_pred_refined = combined_refinement(y_pred, averages)

# Evaluate the refined predictions
print("Accuracy after refinement:", accuracy_score(y_test, y_pred_refined))
print(classification_report(y_test, y_pred_refined))


Accuracy after refinement: 0.8982196531791907
              precision    recall  f1-score   support

           0       0.72      0.92      0.81      2539
           1       0.90      0.21      0.34       485
           2       0.74      0.89      0.81      2437
           3       0.97      0.92      0.94     16164

    accuracy                           0.90     21625
   macro avg       0.83      0.73      0.72     21625
weighted avg       0.91      0.90      0.90     21625



In [ ]:
from sklearn.metrics import classification_report

def classification_report_excluding_class(predictions, true_labels, excluded_class):
    # Filter out the excluded class
    filter_indices = true_labels != excluded_class
    filtered_predictions = predictions[filter_indices]
    filtered_true_labels = true_labels[filter_indices]

    # Generate classification report
    report = classification_report(filtered_true_labels, filtered_predictions, digits=4)

    return report

# Calculate the classification report excluding class 1
report_excluding_1 = classification_report_excluding_class(y_pred_refined, y_test, 1)
print("Classification report excluding class 1:")
print(report_excluding_1)


Classification report excluding class 1:
              precision    recall  f1-score   support

           0     0.7640    0.9153    0.8328      2539
           1     0.0000    0.0000    0.0000         0
           2     0.7771    0.8913    0.8303      2437
           3     0.9695    0.9172    0.9426     16164

    accuracy                         0.9140     21140
   macro avg     0.6276    0.6809    0.6514     21140
weighted avg     0.9227    0.9140    0.9165     21140



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
joblib.dump(model, 'modelv6.joblib')

['modelv6.joblib']

In [ ]:
model = joblib.load('xgboost_golf_swing_model.joblib')

ValueError: ignored

#TEST RADOM FOREST

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer

# Assuming combined_df, train_df, and test_df are already defined
# df = combined_df  # This line seems unnecessary if you're using train_df and test_df

# Define the imputer
imputer = SimpleImputer(strategy='mean')

# Prepare the feature and target variables
X_train = train_df.drop(['event', 'video ID'], axis=1)  # Drop 'event' and 'video ID' for training data
y_train = train_df['event']

X_test = test_df.drop(['event', 'video ID'], axis=1)    # Drop 'event' and 'video ID' for testing data
y_test = test_df['event']

# Apply imputation to handle NaN values
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Initialize RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Predict on the test set
y_pred = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

accuracy, report


(0.8936878612716763,
 '              precision    recall  f1-score   support\n\n           0       0.87      0.70      0.78      2539\n           1       0.81      0.44      0.57       485\n           2       0.75      0.77      0.76      2437\n           3       0.92      0.96      0.94     16164\n\n    accuracy                           0.89     21625\n   macro avg       0.84      0.72      0.76     21625\nweighted avg       0.89      0.89      0.89     21625\n')